In [121]:
import requests
from bs4 import BeautifulSoup
import json

# URL của trang web cần lấy dữ liệu
url = "https://www.presight.io/privacy-policy.html"

# Gửi yêu cầu GET đến trang web
response = requests.get(url)
response.raise_for_status() 

# Phân tích cú pháp nội dung HTML của trang
soup = BeautifulSoup(response.content, 'html.parser')

# Trích xuất nội dung và cấu trúc thành định dạng JSON
privacy_content = []
current_header = None
current_subheader = None
header_classes = None
subheader_classes = None  # Lưu trữ lớp CSS của tiêu đề phụ hiện tại

for tag in soup.find_all(['h2', 'p', 'i', 'li']):
    tag_classes = tag.get("class", [])

    if tag.name == 'h2':
        # Kiểm tra xem thẻ h2 hiện tại có cùng lớp với tiêu đề phụ hiện tại không
        if current_subheader and tag_classes == subheader_classes:
            # Nếu cùng lớp, coi thẻ h2 này là tiêu đề phụ
            current_subheader = {
                "subheader": tag.get_text(strip=True),
                "class": tag_classes,
                "items": []
            }
            current_header["sub_content"].append(current_subheader)
        elif current_header and not current_header["sub_content"] and header_classes != tag_classes:
            # Nếu tiêu đề hiện tại không có sub_content và lớp CSS khác nhau, coi thẻ h2 là tiêu đề phụ
            current_subheader = {
                "subheader": tag.get_text(strip=True),
                "class": tag_classes,
                "items": []
            }
            current_header["sub_content"].append(current_subheader)
        else:
            # Thêm một tiêu đề chính mới
            current_header = {
                "header": tag.get_text(strip=True),
                "class": tag_classes,
                "sub_content": []
            }
            header_classes = tag_classes
            privacy_content.append(current_header)
            current_subheader = None  # Đặt lại tiêu đề phụ cho tiêu đề mới

    elif tag.name == 'i' and current_header is not None:
        # Thêm một tiêu đề phụ mới nếu lớp CSS phù hợp hoặc nó là một tiêu đề phụ độc lập
        current_subheader = {
            "subheader": tag.get_text(strip=True),
            "class": tag_classes,
            "items": []
        }
        current_header["sub_content"].append(current_subheader)
        subheader_classes = tag_classes  # Cập nhật lớp CSS của tiêu đề phụ hiện tại

    elif tag.name in ['p', 'li'] and current_header is not None:
        # Thêm nội dung vào tiêu đề phụ hoặc tiêu đề chính hiện tại
        content_text = tag.get_text(strip=True)
        if current_subheader is not None:
            current_subheader["items"].append(content_text)
        else:
            current_header["sub_content"].append(content_text)

    else:
        # Xử lý nội dung không có tiêu đề
        privacy_content.append({
            "header": None,
            "sub_content": [tag.get_text(strip=True)]
        })


output_filename = "privacy_policy.json"

# Lưu nội dung vào tệp JSON
with open(output_filename, "w", encoding="utf-8") as file:
    json.dump(privacy_content, file, ensure_ascii=False, indent=4)

# Câu lệnh print được cải tiến với tên tệp mới
print(f"Crawl dữ liệu thành công! Dữ liệu đã được lưu vào tệp '{output_filename}'.")


Crawl dữ liệu thành công! Dữ liệu đã được lưu vào tệp 'privacy_policy.json'.
